In [1]:
import os
import sys
import cartopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from scipy.interpolate import griddata
from matplotlib.colors import LogNorm, LinearSegmentedColormap
from matplotlib.lines import Line2D
from datetime import datetime
from cartopy import crs as ccrs
import cartopy.io.img_tiles as cimgt 

In [2]:
# Variable Selection
date_selection = '2024-05-05'
area = {
    'US':[-128,-65,22,52]
    }
region = 'US'
set_extent = area.get(region)
root_folder = f'{os.path.dirname(sys.path[0])}/data/get_states/{date_selection}'

In [3]:
def read_csv(root_folder,csv_file):
    df = pd.read_csv(f'{root_folder}/csv/{csv_file}') 
    df = df[df['on_ground'] == False] #Excluding Taxiing Aircraft
    df['x_speed'] = np.sin(np.deg2rad(df['true_track']))*df['velocity']
    df['y_speed'] = np.cos(np.deg2rad(df['true_track']))*df['velocity']
    df_aircraft = [(x,y,u,v,speed,country) for x,y,u,v,speed,country in zip(df['longitude'],df['latitude'],df['x_speed'],df['y_speed'],df['velocity'],df['origin_country'])]
    return df_aircraft

In [4]:
def read_airports():
    airport_path = f'{os.path.dirname(sys.path[0])}/data/misc_dims/top_us_airport.csv'
    df_airport = pd.read_csv(airport_path) 
    return df_airport

In [5]:
def map_tile_request():
    with open(f'/home/filpill/work/flight_tracking/scripts/keys/stadiamaps.key', 'r') as f:
        creds = f.read().strip().split(' ')
        API_KEY = creds[0]
    os.environ['API_KEY'] = API_KEY
    map_tile = cimgt.StadiaMapsTiles(apikey=API_KEY,style='alidade_smooth_dark',resolution="@2x") 
    return map_tile 
map_tile = map_tile_request()

In [99]:
def base_chart(map_tile,set_extent):
    
    # Figure Size, Chart Elements and Geographic Coordinate Projection
    fig = plt.figure(figsize =(16,9))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Retrieving airport and plotting coordinates
    alpha = 0.7 # Set marker transparency
    df_airport = read_airports()
    ax.plot(
        df_airport['lon'],df_airport['lat'],transform=ccrs.PlateCarree(),
        color='purple',ls='',marker='o', markersize=20, markerfacecolor='none', markeredgewidth=3,
        alpha=alpha
    )
    # Labelling Airport with IATA Codes
    [r,g,b] = [0.5,0.1,0.6]
    [lw,pad] = [0.2,1.2]
    for iata,lon,lat in zip(df_airport['iata'],df_airport['lon'],df_airport['lat']):
        ax.text(
            lon,lat-1,iata,
            transform=ccrs.PlateCarree(),
            color='white',
            fontsize=10,
            weight='bold',
            bbox=dict(fc=(r,g,b), lw=lw, pad=pad,alpha=alpha),
            va='top',ha='center',
            alpha=alpha+0.25
        )


    # List available styles in a Dictionary
    map_styles = {
        1:'default',
        2:'alidade-dark'
    }
    map_select = map_styles.get(1)

    # Logic to Draw the map selected from Dictionary
    if map_select == 'default':
        ax.add_feature(cartopy.feature.LAND)
        ax.add_feature(cartopy.feature.OCEAN)
        ax.add_feature(cartopy.feature.COASTLINE,linewidth=0.3)
        ax.add_feature(cartopy.feature.BORDERS,linestyle='-',linewidth=0.3)
        ax.add_feature(cartopy.feature.LAKES,alpha=0.1)
        ax.add_feature(cartopy.feature.RIVERS,alpha=0.2)
        ax.coastlines()
        ax.set_extent(set_extent, crs=ccrs.PlateCarree()) 
        ax.text(
            s='Created By Filip Livancic', 
            x=0.00675, y=0.0145, transform=ax.transAxes, 
            bbox=dict(facecolor='white',alpha=0.65), fontsize=7,                                    
            va='bottom', ha='left',
        )
        
    elif map_select == 'alidade-dark':
        ax.add_image(map_tile,6,interpolation='spline36')
        for y_pos, s_text in zip([0.0145,0.0545],['Created By Filip Livancic','Map Data: ©Stadia Maps ©OpenMapTiles ©OpenStreetMap']):
            ax.text(
                s=s_text, 
                x=0.00675, y=y_pos, transform=ax.transAxes, 
                bbox=dict(facecolor='white',alpha=0.65), fontsize=7,
                va='bottom', ha='left',
            )
    return fig, ax

In [37]:
def filter_data_boundary(df_aircraft,set_extent):
    # Filtering Data To Desired Airspace Boundary
    df_aircraft = pd.DataFrame(df_aircraft,columns=['x','y','u','v','speed','country'])
    df_aircraft_filtered = df_aircraft[
                                      (df_aircraft['x']>=set_extent[0]) &
                                      (df_aircraft['x']<=set_extent[1]) &
                                      (df_aircraft['y']>=set_extent[2]) &
                                      (df_aircraft['y']<=set_extent[3])
                                    ] 
    return df_aircraft_filtered

In [38]:
def kde_meshgrid(df_aircraft_filtered,set_extent):
    ac_lat  = df_aircraft_filtered['x']
    ac_long = df_aircraft_filtered['y']
    xmin = set_extent[0]
    xmax = set_extent[1]
    ymin = set_extent[2]
    ymax = set_extent[3]
    X, Y = np.mgrid[xmin:xmax:175j, ymin:ymax:175j]
    positions = np.vstack([X.ravel(), Y.ravel()])
    values = np.vstack([ac_lat, ac_long])
    kernel = gaussian_kde(values)
    density = np.reshape(kernel(positions).T, X.shape)
    return density

In [39]:
# Config For Plotting Various Geolocations
plot_config = { 
    "regions" : {
        "eu": {
            "title":"EU Region",
            "set_extent":[-15,30,35,60],
            "headwidth":10,
            "headaxislength":7,
            "headlength":9,
            "width":0.001,
            "scale":20000,
            "x":0.9955,
            "y":0.0085
        },
        "us": {
            "title":"US Region",
            "set_extent":[-128,-65,22,52], 
            "headwidth":10,            
            "headaxislength":5,
            "headlength":8,
            "width":0.0025,
            "scale":35000,
            "x":0.9955,
            "y":0.0095
        }
    }
}
region_config = plot_config['regions']['us']

In [101]:
def plot_kde(
        df_aircraft_filtered,
        root_folder,
        timestamp,
        region,
        fig,
        ax,
        density,
        region_config
    ):
    fig=fig
    ax=ax
    ax.imshow(np.rot90(density), cmap=plt.cm.turbo,
    extent=[set_extent[0], set_extent[1], set_extent[2], set_extent[3]])
    arrow = ax.quiver(
        df_aircraft_filtered['x'],
        df_aircraft_filtered['y'],
        df_aircraft_filtered['u'],
        df_aircraft_filtered['v'],
        transform=ccrs.PlateCarree(), 
        headwidth=region_config.get('headwidth'),
        headaxislength=region_config.get('headaxislength'),
        headlength=region_config.get('headlength'),
        width=region_config.get('width'),
        scale=region_config.get('scale'),
        color='white',
        edgecolor='black',
        linewidth=0.35,
        alpha=0.85
    )
    ax.set_xlim([set_extent[0], set_extent[1]])
    
    title = ax.set_title(
        f'OpenSky Network API | Aircraft Tracking - {region} Region | {timestamp}',
        loc='left', 
        fontsize=10
    )
    ax.figure.savefig(f'{root_folder}/gaussian_kde/{timestamp}.png', format='png', dpi=200)

    # Clear arrows before returning plot back to main script
    arrow.remove()
    return fig,ax

In [41]:
def extract_timestamp(csv_file):
    timestamp = csv_file.split('.')[0]
    timestamp = datetime.strptime(timestamp,'%Y-%m-%dT%H:%M:%S')
    timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S UTC')
    return timestamp

In [103]:
# Create Base Chart with only Map and No Datapoints
fig, ax = base_chart(map_tile,set_extent)
# Looping Through Each CSV, Plotting Scatterpoints And Saving Chart
for csv_file in sorted(os.listdir(f'{root_folder}/csv')):
    print(f'Creating Image from: {csv_file}')
    timestamp = extract_timestamp(csv_file)
    df_aircraft = read_csv(root_folder,csv_file)
    df_aircraft_filtered = filter_data_boundary(df_aircraft,set_extent)
    density = kde_meshgrid(df_aircraft_filtered,set_extent)
    fig, ax = plot_kde(
        df_aircraft_filtered,
        root_folder,
        timestamp,
        region,
        fig,
        ax,
        density,
        region_config
    )

Creating Image from: 2024-05-05T20:14:36.539206.csv
Creating Image from: 2024-05-05T20:15:38.577825.csv
Creating Image from: 2024-05-05T20:16:40.377785.csv
Creating Image from: 2024-05-05T20:17:42.192539.csv
Creating Image from: 2024-05-05T20:18:43.751990.csv
Creating Image from: 2024-05-05T20:19:45.204410.csv
Creating Image from: 2024-05-05T20:20:49.604984.csv
Creating Image from: 2024-05-05T20:21:51.412865.csv
Creating Image from: 2024-05-05T20:22:53.423664.csv
Creating Image from: 2024-05-05T20:23:55.012803.csv
Creating Image from: 2024-05-05T20:24:56.335714.csv
Creating Image from: 2024-05-05T20:25:58.945417.csv
Creating Image from: 2024-05-05T20:27:00.336425.csv
Creating Image from: 2024-05-05T20:28:03.699792.csv
Creating Image from: 2024-05-05T20:29:06.833415.csv
Creating Image from: 2024-05-05T20:30:09.471407.csv
Creating Image from: 2024-05-05T20:31:14.980330.csv
Creating Image from: 2024-05-05T20:32:16.639509.csv
Creating Image from: 2024-05-05T20:33:18.279394.csv
Creating Ima